In [1]:
import warnings
warnings.filterwarnings("ignore")

from data_loading import *

from xai_agg.agg_exp import *
from xai_agg.utils import *

import pandas as pd
import numpy as np

from scipy.stats import pearsonr, spearmanr, kendalltau

2025-01-18 18:50:38.342881: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-18 18:50:38.366852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset_name, preprocessed_data, categorical_features, X, y, X_train, X_test, y_train, y_test, clf = load_pakdd2010_data()

Accuracy: 0.7466666666666667
ROC AUC: 0.5158014399393709


In [3]:
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
evaluator.init()

Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1698 - val_loss: 1.0884
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - loss: 1.2200 - val_loss: 1.0398
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - loss: 1.1136 - val_loss: 0.9910
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - loss: 1.1491 - val_loss: 0.9464
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 1.0467 - val_loss: 0.9088
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 0.9962 - val_loss: 0.8787
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - loss: 1.0078 - val_loss: 0.8549
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 1.0518 - val_loss: 0.8365
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 0.9414 - val_loss: 0.8224
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 1.0205 - val_loss: 0.8117
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - loss: 1.0061 - val_loss: 0.8032
Epoch 12/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 

### Misc

In [4]:
evaluator.faithfullness_correlation(ShapTabularTreeWrapper, X_test.iloc[0])

0.6798262974894057

# Generating metric vectors and calculating correlation between them

In [5]:
# Getting a few reanknings
exps = [LimeWrapper(clf, X_train, categorical_features), ShapTabularTreeWrapper(clf, X_train, categorical_features), AnchorWrapper(clf, X_train, categorical_features)]

indexes = np.random.choice(X_test.index, 100, replace=False)


In [6]:
import concurrent.futures

comp_arr = [] # original, new1, new2, ...

def process_index(idx):
    local_comp_arr = []
    for exp in exps:
        row = []
        explanation = exp.explain_instance(X_test.loc[idx])
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, iterations=100, len_subset=1))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, iterations=100, len_subset=1))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="sum", iterations=100, len_subset=1))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="percentile", iterations=100, len_subset=1))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="avg", iterations=100, len_subset=1))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="inverse", iterations=100, len_subset=1))
        
        # row.append(evaluator.complexity(exp, X_test.loc[idx], explanation=explanation))
        # row.append(evaluator.nrc(exp, X_test.loc[idx], explanation=explanation))
        
        local_comp_arr.append(row)
    return local_comp_arr
    
with concurrent.futures.ProcessPoolExecutor(5) as executor:
    results = list(executor.map(process_index, indexes))

for result in results:
    comp_arr.extend(result)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

In [7]:
comp_arr

[[0.11853072900427095,
  0.1363129402332447,
  0.22931389854833853,
  0.13767670846576102,
  0.18690747596967347,
  0.23892763743694703],
 [0.49614895884776533,
  0.6370574852126564,
  0.5242253048981342,
  0.48144955150066454,
  0.650587077373043,
  0.2858447016662469],
 [0.24361441588445976,
  0.2657370872385186,
  0.060028358820672485,
  0.03908218128636823,
  0.039745791091461674,
  0.10982513972477666],
 [0.5688897171793269,
  0.5406401056495008,
  0.20445491509763636,
  0.2973614261818426,
  0.2516792443105751,
  0.49737282541520594],
 [0.19421995776279516,
  0.23865531496465953,
  0.54063228052811,
  0.3225396873360568,
  0.47197246990570396,
  0.035562093058555],
 [0.4522015331777408,
  0.6170251797739762,
  0.3117029079183893,
  0.36885231721707534,
  0.3864003841725882,
  0.06298818457351206],
 [0.5840135795125212,
  0.5063126138245544,
  0.5589343080776064,
  0.481525880538293,
  0.25727644488600054,
  0.7273977862818175],
 [0.5628833069866699,
  0.4709182680404334,
  0.6377

In [8]:
from scipy.stats import spearmanr

comp_arr = np.array(comp_arr)
rank_algs = ["faithfulness (itself)", "rank_based_fairthfulness:sum", "rank_based_fairthfulness:percentile", "rank_based_fairthfulness:avg", "rank_based_fairthfulness:inverse"]
# rank_algs = ["NRC"]

pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]
kendall_corr = [kendalltau(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

correlation_df = pd.DataFrame({
    'Rank Algorithm': rank_algs,
    'Pearson Correlation': pearson_corr,
    'Spearman Correlation': spearman_corr,
    'Kendall Correlation': kendall_corr
})

print(dataset_name)
print("The following are the correlations between the faithfullness metric and... (abs fixed, subset_len=1, iterations=100)")
display(correlation_df)

PAKDD2010
The following are the correlations between the faithfullness metric and... (abs fixed, subset_len=1, iterations=100)


,Rank Algorithm,Pearson Correlation,Spearman Correlation,Kendall Correlation
0,faithfulness (itself),0.847137,0.841800,0.662609
1,rank_based_fairthfulness:sum,0.675876,0.666648,0.479331
2,rank_based_fairthfulness:percentile,0.684090,0.672473,0.480045
3,rank_based_fairthfulness:avg,0.673816,0.664134,0.475630
4,rank_based_fairthfulness:inverse,0.802265,0.801084,0.601338
